In [2]:
import pandas as pd
import numpy as np
from setfit import SetFitModel, TrainingArguments
from datasets import load_dataset, Dataset, DatasetDict
from setfit import Trainer

all_valids = pd.read_csv('../data/train_valid_all.csv')
print(all_valids.shape)

all_valids = all_valids[['Response', 'Label']]

# Split all_valids into train, test, and validation datasets
train_samples = all_valids.groupby('Label').apply(lambda x: x.sample(5)).reset_index(drop=True)
remaining_samples = all_valids.drop(train_samples.index).reset_index(drop=True)
test_samples = remaining_samples.groupby('Label').apply(lambda x: x.sample(frac=0.67)).reset_index(drop=True)
validation_samples = remaining_samples.drop(test_samples.index).reset_index(drop=True)


(678, 10)


In [3]:
print("train shape: ", train_samples.shape)
print("test shape: ", test_samples.shape)
print("validation shape: ", validation_samples.shape)

train_dataset = Dataset.from_pandas(train_samples)
val_dataset = Dataset.from_pandas(validation_samples)
test_dataset = Dataset.from_pandas(test_samples)

train shape:  (25, 2)
test shape:  (437, 2)
validation shape:  (216, 2)


In [4]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['Response', 'Label'],
        num_rows: 25
    })
    validation: Dataset({
        features: ['Response', 'Label'],
        num_rows: 216
    })
    test: Dataset({
        features: ['Response', 'Label'],
        num_rows: 437
    })
})

In [5]:

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
model.labels = [1, 2, 3, 4, 5]

train_dataset = dataset_dict["train"]
test_dataset = dataset_dict["test"]
val_dataset = dataset_dict["validation"]




model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [6]:
train_dataset 

Dataset({
    features: ['Response', 'Label'],
    num_rows: 25
})

In [16]:
args = TrainingArguments(
    batch_size=8,
    num_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    column_mapping={"Response": "text", "Label": "label"}
)


Applying column mapping to the training dataset
/Users/kaylahuang/opt/anaconda3/envs/mercor/lib/python3.12/site-packages/datasets/utils/_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [17]:
trainer.train()

***** Running training *****
  Num unique pairs = 500
  Batch size = 8
  Num epochs = 1
  Total optimization steps = 63


  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]